<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from gensim.models import KeyedVectors
import numpy as np
import os

#Load model

Tải pretrained model cho Tiếng Việt

In [ ]:
!pip install fasttext


  Using cached https://files.pythonhosted.org/packages/f8/85/e2b368ab6d3528827b147fdb814f8189acc981a4bc2f99ab894650e05c40/fasttext-0.9.2.tar.gz
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3096966 sha256=e34062d73b0465b928102841107a3593fbbf0056f9e8611f7eafe78cfb2127f7
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext.util


In [ ]:
fasttext.util.download_model('vi',if_exists='ignore')

Load

In [ ]:
ft = fasttext.load_model('/content/drive/MyDrive/DeepLearning_Application/Lab3-Word2Vec-With-Pretrained-Model/cc.vi.300.bin')

In [3]:
%cd /content/drive/MyDrive/DeepLearning_Application/Assignment3-SentimentAnalysis-with-LSTM/

/content/drive/MyDrive/DeepLearning_Application/Assignment3-SentimentAnalysis-with-LSTM


#Load tập từ vựng và ma trận word-embedding

In [4]:
currentDir = '/content/drive/MyDrive/DeepLearning_Application/Assignment3-SentimentAnalysis-with-LSTM/'
wordsList = np.load(os.path.join(currentDir, 'wordsList.npy'))
print('Simplified vocabulary loaded!')
wordsList = wordsList.tolist()

wordVectors = np.load(os.path.join(currentDir, 'wordVectors.npy'))
wordVectors = np.float32(wordVectors)
print ('Word embedding matrix loaded!')

Simplified vocabulary loaded!
Word embedding matrix loaded!


In [ ]:
print('Size of the vocabulary: ', len(wordsList))
print('Size of the word embedding matrix: ', wordVectors.shape)

Size of the vocabulary:  19899
Size of the word embedding matrix:  (19899, 300)


In [ ]:
import tensorflow as tf
maxSeqLength = 10   #Maximum length of sentence
numDimensions = 300 #Dimensions for each word vector
sentenceIndexes = np.zeros((maxSeqLength), dtype='int32')

# TODO 3.1: Gán chỉ số của các từ trong câu và 'sentenceIndexes'
sentence = 'Món này ăn hoài không hề biết chán'
words = sentence.split(" ")
for index,word in enumerate(words):
    word_idx = wordsList.index(word.lower())
    sentenceIndexes[index]=word_idx
print(sentenceIndexes)
#  Ma trận biểu diễn:
print('Sentence representation of word vectors:')
print(tf.nn.embedding_lookup(wordVectors,sentenceIndexes).shape)

[  119  8136  4884 18791 16614 12231 15951  3371     0     0]
Sentence representation of word vectors:
(10, 300)


Visual Dataset

In [ ]:
from os import listdir
from os.path import isfile, join
positiveFiles = ['positiveReviews/' + f for f in listdir('positiveReviews/') if isfile(join('positiveReviews/', f))]
negativeFiles = ['negativeReviews/' + f for f in listdir('negativeReviews/') if isfile(join('negativeReviews/', f))]
numWords = []
for pf in positiveFiles:
    with open(pf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)       
print('Positive files finished')

for nf in negativeFiles:
    with open(nf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)  
print('Negative files finished')

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))

#Chuẩn hóa văn bản và tách từ

In [6]:
import re
strip_special_chars = re.compile("[^\w0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [65]:
maxSeqLength = 180

Chạy và lưu vào file idsMatrix.npy

In [ ]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
nFiles = 0
# Index of Unknow word
unk_idx = wordsList.index('UNK')

for pf in positiveFiles:
    with open(pf, "r", encoding="utf-8") as f:
        nIndexes = 0
        line=f.readline()
        cleanedLine = cleanSentences(line)
        split = cleanedLine.split()
        for word in split:
            # TODO 3.2: Nếu 'word' thuộc tập 'wordsList' thì gán chỉ số của 'word' vào ma trận ids
            if word in wordsList:
               word_idx = wordsList.index(word)
               ids[nFiles][word_idx] = word_idx
            # Ngược lại: gán 'unk_idx' vào ma trận ids
            nIndexes = nIndexes + 1
            if nIndexes >= maxSeqLength:
                break
        nFiles = nFiles + 1 

print('Positive files are indexed!')
for nf in negativeFiles:
    with open(nf, "r", encoding="utf-8") as f:
        nIndexes = 0
        line=f.readline()
        cleanedLine = cleanSentences(line)
        split = cleanedLine.split()
        for word in split:
            # ToDo 3.2: tương tự như trên. Không khác gì hết.
            if word in wordsList:
               word_idx = wordsList.index(word)
               ids[nFiles][word_idx] = word_idx
            nIndexes = nIndexes + 1
            if nIndexes >= maxSeqLength:
                break
        nFiles = nFiles + 1 

print('Negative files are indexed!')
# Save ids Matrix for future uses.
np.save(os.path.join(currentDir,'idsMatrix.npy'), ids)

Load file đã chạy để tiết kiệm thời gian

In [7]:
ids = np.load(os.path.join(currentDir,'idsMatrix.npy'))
print('Word indexes of the first review: ', ids[0])


Word indexes of the first review:  [19898  1906  4454  5284 10661 11694 11994 18784 18569 18619 13174  9821
 14794  8884  6443  5767  8589 18850 15570  5596   799 11060  4222 16893
 13078  8136  3364  4454  4756 10304  8885  3553  9782  1232 14359 10606
   579 15522  2219 15092 14855 15253  4884  3364  5519  4558  9649   269
 15522 12309 14855 11503  2212  4884  7155 11577  4222  5767 15076 12225
 10774  1218  2876 19584  4558  2974 13452  5013   842 10642 17292 11895
   803 11060 16760  1906 15253 14598 15253  1047  5668  4884 10642 12225
  7090 17292 18109 13078 16334  1238  3364  5519  4135  3553 14967  4964
 15385  9673  2997 14855  7446  8038 11440  1345   842  5767   803 11060
 18791  5013     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0    

#Xây dựng hàm lấy dữ liệu train và test theo từng batch

In [63]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        if (i % 2 == 0): 
            # Pick positive samples randomly
            num = randint(1,13999)
            labels.append([1,0])
        else:
            # Pick negative samples randomly
            num = randint(15999,29999)
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(13999,15999)
        if (num <= 14999):
            labels.append([1,0])
        else:
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

In [66]:
x_train,y_train = getTrainBatch()

In [69]:
x_train

array([[12645.,  5596.,  1868., ...,     0.,     0.,     0.],
       [11975.,  5284.,  8591., ...,     0.,     0.,     0.],
       [12645.,  5596., 19492., ...,     0.,     0.,     0.],
       ...,
       [ 2326., 19492., 10411., ...,     0.,     0.,     0.],
       [ 6874., 18543.,  4756., ...,     0.,     0.,     0.],
       [ 5496., 11694.,  2673., ...,     0.,     0.,     0.]])

#3.Xây dựng RNN Model với Tensorflow

In [11]:
# Initialize paramters
numDimensions = 300
batchSize = 64
lstmUnits = 128
nLayers = 2
numClasses = 2
iterations = 30000

In [8]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers

In [92]:
model = keras.Sequential([
        keras.Input(shape=(batchSize,180)),
        layers.LSTM(lstmUnits,return_sequences=True),
        layers.Dropout(0.5),
        layers.LSTM(lstmUnits),
        layers.Dense(2,activation='sigmoid')]
 )

In [93]:
input = np.zeros((64,180))
model.predict(input)

ValueError: ignored

In [62]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 64, 128)           158208    
_________________________________________________________________
dropout (Dropout)            (None, 64, 128)           0         
_________________________________________________________________
lstm_45 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 258       
Total params: 290,050
Trainable params: 290,050
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='Adam',
    metrics=["accuracy"])
 

In [90]:
x_train = np.concatenate((ids[0:14000],ids[16000:30000]),axis=0)
y_train = np.ones((28000,1))
x_test = np.concatenate((ids[14000:15000],ids[15000:16000]),axis=0)
y_test = np.ones((2000,1))

In [91]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batchSize, epochs=iterations)

Epoch 1/30000


ValueError: ignored